In [24]:
import base64
import requests
TEST_NUM=101
# OpenAI API Key
api_key =''
headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}
# Function to encode the image
responseList=[]
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

for i in range(TEST_NUM):
	if i!=87:
	# Path to your image
		image_path = f"../vega_ans/chart{i}.png"
		# Getting the base64 string
		base64_image = encode_image(image_path)
		payload = {
		"model": "gpt-4o-mini",
		"messages": [
			{
			"role": "user",
			"content": [
				{
				"type": "text",
				"text": "Given a chart image, list data facts from general to detailed. The following are some data fact examples of different kinds. #COMPARE: The ticket price shown for 2006 is just over 60 dollars, and it rises from that point to just over 100 dollars in 2018. #TREND: Viewers of Minecraft on twitch has gradually increased between 2018 and 2020. #RANGE: Consumption was fairly stable between 2000 and 2005. #FILTER: The majorly of the big cities are higer than 3 index points.#RETRIEVE: Omsk has the largest value of 3.5 index points. Please briefly respond without any # tags, and each sentence should end with a period:"
				},
				{
				"type": "image_url",
				"image_url": {
					"url": f"data:image/jpeg;base64,{base64_image}",
					"detail": "low"
				}
				}
			]
			}
		],
		"max_tokens": 200, "temperature": 0
		}

		response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

		responseList.append(response.json()['choices'][0]['message']['content'])

In [ ]:
responseList

In [ ]:
import pickle
import json
with open ('../outlist.pkl','rb') as f:
    output_list=pickle.load(f)
answerList=[]
with open ('../data_test.json','r') as f:
    test_dataset=json.load(f)
for index,ouput in enumerate(output_list):
    data=test_dataset[index]
    answerList.append(data['caption_L2L3'])
answerList[:5]

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
scoreList=[]
for i in range(TEST_NUM):
	if i>87:
		query_embedding = model.encode(responseList[i-1])
		passage_embedding = model.encode(answerList[i])
	else:
		query_embedding = model.encode(responseList[i])
		passage_embedding = model.encode(answerList[i])
	scoreList.append(float(util.dot_score(query_embedding, passage_embedding)))
len(scoreList)

In [ ]:
import numpy as np
np.average(scoreList[:100])

In [34]:
with open('gpt-score.pkl', 'wb') as file:
    pickle.dump(scoreList, file)